# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [28]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*') ]

In [29]:
len(archivos)

238

In [30]:
archivos[0]

WindowsPath('datos/pycon-ar-2011/videos/.ipynb_checkpoints')

- Importar los datos con pandas

- Realizar graficos